In [ ]:
# # Deterministic Pipeline Test on Static Images
# This notebook tests the deterministic algorithm (overlap and slant calculation) by loading static images of the aluminum baseline and the glass slide.

# mports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os


# Add the src directory to the path so we can import local modules
sys.path.append(os.path.abspath('../src'))

# Local Imports based on provided files
import Deterministic_Predict as det
import Detect_Glass as glass_det
import Detect_Alum as alum_det
import Image_Proccessing as im_proc
from shapely.geometry import Polygon

In [ ]:
# --- Configuration ---
# Define the hardcoded crop windows used in the main execution script
# Format: (x_start, y_start, width, height)
INTERNAL_CROP_RECT_TOP = (900, 260, 380, 250)
INTERNAL_CROP_RECT_BOT = (900, 540, 380, 250)

# Choose which position to test
CURRENT_CROP = INTERNAL_CROP_RECT_TOP

# Define thresholds for success (taken from main script example values)
OVERLAP_THRESHOLD = 0.9
SLANT_THRESHOLD = 0.5 # Note: In main script it checked 'slant > threshold', which seems wrong for a difference calculation. Usually lower difference is better. Assuming lower is better here.

# --- Helper Visualization Function ---
def display_image_with_box(image_bgr, corners, title="Image"):
    """Helper to draw bounding box and display using matplotlib"""
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    if corners is not None:
        # Convert tuple list to numpy array for polylines
        pts = np.array(corners, np.int32)
        pts = pts.reshape((-1, 1, 2))
        # Draw green box for detection
        cv2.polylines(img_rgb, [pts], True, (0, 255, 0), 3)

    plt.figure(figsize=(10, 6))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

# --- Static Processing Adapter ---
# The original code uses 'repeated_sample' for live feeds.
# We create an adapter for a single static image.
def process_static_image(image_path, detector_obj, crop_window):
    frame_bgr = cv2.imread(image_path)
    if frame_bgr is None:
        print(f"Error: Could not load image at {image_path}")
        return None, None

    # Find corners using the specific detector module (alum or glass)
    corners = detector_obj.find_corners(frame_bgr, crop_window)
    return frame_bgr, corners

# %% [markdown]
# ## 2. Load and Process Aluminum Baseline
# We load an image of the aluminum carrier without glass to establish the baseline area.

# %%
alum_image_path = '../test_images/aluminum_baseline.jpg' # UPDATE THIS PATH based on your setup

print(f"Processing Aluminum Baseline: {alum_image_path}...")
alum_img, alum_corners = process_static_image(alum_image_path, alum_det, CURRENT_CROP)

if alum_corners is not None:
    print("Aluminum detected successfully.")
    display_image_with_box(alum_img, alum_corners, title="Aluminum Baseline Detection")
else:
    print("Failed to detect aluminum in baseline image.")

# %% [markdown]
# ## 3. Load and Process Glass Test Image
# We load an image containing the glass slide to compare against the baseline.

# %%
glass_image_path = '../test_images/glass_success.jpg' # UPDATE THIS PATH to test different images

print(f"Processing Glass Image: {glass_image_path}...")
glass_img, glass_corners = process_static_image(glass_image_path, glass_det, CURRENT_CROP)

if glass_corners is not None:
    print("Glass detected successfully.")
    display_image_with_box(glass_img, glass_corners, title="Glass Detection")
else:
    print("Failed to detect glass in test image.")

# %% [markdown]
# ## 4. Calculate Deterministic Metrics
# Using the corners found above, we calculate overlap percentage and slant difference using functions directly from `Deterministic_Predict.py`.

# %%
if alum_corners is not None and glass_corners is not None:
    # Calculate Overlap
    # Note: We call compare_rectangles directly on the single result, not using averaged_sampling
    overlap_percent = det.compare_rectangles(alum_corners, glass_corners)

    # Calculate Slant Difference
    slant_difference = det.slant_diff(alum_corners, glass_corners)

    print("-" * 30)
    print(f"Results for {os.path.basename(glass_image_path)}")
    print("-" * 30)
    print(f"Overlap Percentage: {overlap_percent:.2f}%")
    print(f"Slant Difference:   {slant_difference:.4f}")
    print("-" * 30)

    # Determine Success based on thresholds
    # Note: Adjusting logic based on assumption that lower slant difference is better
    is_overlap_good = overlap_percent > (OVERLAP_THRESHOLD * 100) # Convert threshold to percent
    is_slant_good = slant_difference < SLANT_THRESHOLD

    if is_overlap_good and is_slant_good:
        print("Final Prediction: SUCCESS")
    else:
        print(f"Final Prediction: FAILURE (Overlap OK: {is_overlap_good}, Slant OK: {is_slant_good})")

else:
    print("Cannot calculate metrics: Detection failed for either aluminum or glass.")